In [1]:
from sympy import *
from sympy.abc import *

In [2]:
x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11 = symbols('x:12')

In [3]:
to_cnf((x0 & x1) | (x2 & x3) | (x4 & x5) | (x6 & x7)| (x8 & x9)| (x10 & x11))

(x0 | x10 | x2 | x4 | x6 | x8) & (x0 | x10 | x2 | x4 | x6 | x9) & (x0 | x10 | x2 | x4 | x7 | x8) & (x0 | x10 | x2 | x4 | x7 | x9) & (x0 | x10 | x2 | x5 | x6 | x8) & (x0 | x10 | x2 | x5 | x6 | x9) & (x0 | x10 | x2 | x5 | x7 | x8) & (x0 | x10 | x2 | x5 | x7 | x9) & (x0 | x10 | x3 | x4 | x6 | x8) & (x0 | x10 | x3 | x4 | x6 | x9) & (x0 | x10 | x3 | x4 | x7 | x8) & (x0 | x10 | x3 | x4 | x7 | x9) & (x0 | x10 | x3 | x5 | x6 | x8) & (x0 | x10 | x3 | x5 | x6 | x9) & (x0 | x10 | x3 | x5 | x7 | x8) & (x0 | x10 | x3 | x5 | x7 | x9) & (x0 | x11 | x2 | x4 | x6 | x8) & (x0 | x11 | x2 | x4 | x6 | x9) & (x0 | x11 | x2 | x4 | x7 | x8) & (x0 | x11 | x2 | x4 | x7 | x9) & (x0 | x11 | x2 | x5 | x6 | x8) & (x0 | x11 | x2 | x5 | x6 | x9) & (x0 | x11 | x2 | x5 | x7 | x8) & (x0 | x11 | x2 | x5 | x7 | x9) & (x0 | x11 | x3 | x4 | x6 | x8) & (x0 | x11 | x3 | x4 | x6 | x9) & (x0 | x11 | x3 | x4 | x7 | x8) & (x0 | x11 | x3 | x4 | x7 | x9) & (x0 | x11 | x3 | x5 | x6 | x8) & (x0 | x11 | x3 | x5 | x6 | x9) & (x0 | x11 

In [4]:
def symbol_init(x, count=0):
    def new_symbol ():
        nonlocal count
        count += 1
        return var(str(x) + str(count))
    return new_symbol

In [5]:
s = symbol_init('t')

In [6]:
s() | s()

t1 | t2

In [7]:
def subformula(expr):
# subformula of expr without atoms
# considering operator as binary
    
    if expr.func == Symbol:
        return []
    elif expr.func == Not:
        return [expr] + subformula(expr.args[0])
    elif ((expr.func == And) or (expr.func == Or)) and len(expr.args)==2:
        return [expr] + subformula(expr.args[0]) + subformula(expr.args[1]) 
    elif ((expr.func == And) or (expr.func == Or)) and len(expr.args)>2:
        return [expr] + subformula(expr.args[0]) + subformula(eval(str(expr.func)+str(expr.args[1:])))  
    else:
        return "error"
        
    

In [8]:
subformula((x1 & x2) | (x3 & x4) | (x5 & x6)| (x7 & x8) )

[(x1 & x2) | (x3 & x4) | (x5 & x6) | (x7 & x8),
 x1 & x2,
 (x3 & x4) | (x5 & x6) | (x7 & x8),
 x3 & x4,
 (x5 & x6) | (x7 & x8),
 x5 & x6,
 x7 & x8]

In [9]:
subformula(Not((A & B ) | C))

[~(C | (A & B)), C | (A & B), A & B]

In [10]:
subformula(Not((A & B & D) | C))

[~(C | (A & B & D)), C | (A & B & D), A & B & D, B & D]

In [11]:
def depth(expr):
# still considering operator as binary
    if expr.func == Not:
        return 1 + depth(expr.args[0])
    elif (((expr.func == And) or (expr.func == Or)) and len(expr.args)==2):
        return 1 + max([depth(x) for x in expr.args])
    elif (((expr.func == And) or (expr.func == Or)) and len(expr.args)>2):
        #return 1 + max(depth(expr.args[0]), depth(eval(str(expr.func)+str(expr.args[1:]))))
        return len(expr.args)-1 + max([depth(x) for x in expr.args])
    elif expr.func == Symbol:
        return 0
    else: 
        return"error"

In [12]:
depth(Not((A & B & D) | Not(C | D)))


4

In [13]:
depth((x1 & x2) | (x3 & x4))

2

In [14]:
depth((x1 & x2 &x3 & x4) )

3

In [15]:
depth((x1 & x2 & x3))

2

In [16]:
sorted(subformula(Not((A & B ) | C)), key=depth)


[A & B, C | (A & B), ~(C | (A & B))]

In [17]:
(x1 & x2 &x3).subs(x1 & x2, A)

x1 & x2 & x3

In [18]:
(x1 & x2 &x3 & x3).args

(x1, x2, x3)

In [19]:
(x1 & Not(x2) &x3).subs(Not(x2) & x3, A)

x1 & x3 & ~x2

In [20]:
def binSubs(x, y, z):
    #replace y by z in x
    #print("subs", x, y, z)
    if (len(y.args) >0) and (len(x.args)>2) and (y!=x) and (y.func==x.func):
     
        if all([y in x.args for y in y.args]): 
            l = list(x.args)
            for i in y.args:
                l.remove(i)
            if (z.func== Symbol):
                t=tuple(l + [z])
            else:
                t=tuple(l+list(z.args))
            return eval(str(x.func) + str(t))
    
    return x.subs(y,z)
            

In [21]:
binSubs((x1 & ~x2 &x3), x1 & ~x2, A)

A & x3

In [22]:
binSubs((x1 & x2 &x3), x1 & x2, A)

A & x3

In [23]:
x =x1 & x2 & x3
y = x1 & x2
z= A
print(z.args)
[y in x.args for y in y.args]
l = list(x.args)
print(l)
for i in y.args:
    l.remove(i)
print(l)
t=tuple(l+list(z.args))
z.func== Symbol
[z]

()
[x1, x2, x3]
[x3]


[A]

In [35]:
E = Not((A & B ) | C)

In [37]:
s = sorted(subformula(E), key=depth)
s

[A & B, C | (A & B), ~(C | (A & B))]

In [38]:
tseitin1(Not((A & B ) | C))

p3 & (Equivalent(p3, ~p2)) & (Equivalent(p1, A & B)) & (Equivalent(p2, C | p1))

In [24]:
def tseitin1(expr):
    # for binary operation

    s=symbol_init("p")
    if expr.func == Symbol:
        return true
    else:
        return tseitin_aux(sorted(subformula(expr), key=depth), s) 

def tseitin_aux(L, s):
    #nonlocal newvar
    newvar = s()
    if (L==[]):
        return true
    if depth(L[0]) > 0:
        if len(L[1:])>0 :
            #newList = [x.subs(L[0], newvar) for x in L[1:]]  
            
            newList = [binSubs(x, L[0], newvar) for x in L[1:]]  
            
            return Equivalent(newvar, L[0]) & tseitin_aux(newList, s)
        else:      
            return Equivalent(newvar, L[0]) & newvar
    else:
        print("error")   

In [25]:
tseitin1(Not((A & B ) | C))

p3 & (Equivalent(p3, ~p2)) & (Equivalent(p1, A & B)) & (Equivalent(p2, C | p1))

In [26]:
to_cnf(tseitin1(Not((A & B ) | C)))

p3 & (p2 | p3) & (A | ~p1) & (B | ~p1) & (p2 | ~C) & (p2 | ~p1) & (~p2 | ~p3) & (C | p1 | ~p2) & (p1 | ~A | ~B)

In [27]:
tseitin1((x1 & x2 & x3))

p2 & (Equivalent(p1, x2 & x3)) & (Equivalent(p2, p1 & x1))

In [28]:
tseitin1((x1 & x2) | (x3 & x4) | (x5 & x6))

p5 & (Equivalent(p1, x1 & x2)) & (Equivalent(p2, x3 & x4)) & (Equivalent(p3, x5 & x6)) & (Equivalent(p4, p2 | p3)) & (Equivalent(p5, p1 | p4))

In [29]:
tseitin1((x1 & x2) | (x3 & x4) | (x5 & x6)| (x7 & x8))

p7 & (Equivalent(p1, x1 & x2)) & (Equivalent(p2, x3 & x4)) & (Equivalent(p3, x5 & x6)) & (Equivalent(p4, x7 & x8)) & (Equivalent(p5, p3 | p4)) & (Equivalent(p6, p2 | p5)) & (Equivalent(p7, p1 | p6))

In [30]:
to_cnf(tseitin1((x1 & x2) | (x3 & x4) | (x5 & x6)| (x7 & x8)))

p7 & (p5 | ~p3) & (p5 | ~p4) & (p6 | ~p2) & (p6 | ~p5) & (p7 | ~p1) & (p7 | ~p6) & (x1 | ~p1) & (x2 | ~p1) & (x3 | ~p2) & (x4 | ~p2) & (x5 | ~p3) & (x6 | ~p3) & (x7 | ~p4) & (x8 | ~p4) & (p1 | p6 | ~p7) & (p2 | p5 | ~p6) & (p3 | p4 | ~p5) & (p1 | ~x1 | ~x2) & (p2 | ~x3 | ~x4) & (p3 | ~x5 | ~x6) & (p4 | ~x7 | ~x8)

In [31]:
tseitin1((x0 & x1) | (x2 & x3) | (x4 & x5) | (x6 & x7)| (x8 & x9)| (x10 & x11))

p11 & (Equivalent(p1, x0 & x1)) & (Equivalent(p10, p2 | p9)) & (Equivalent(p11, p1 | p10)) & (Equivalent(p2, x10 & x11)) & (Equivalent(p3, x2 & x3)) & (Equivalent(p4, x4 & x5)) & (Equivalent(p5, x6 & x7)) & (Equivalent(p6, x8 & x9)) & (Equivalent(p7, p5 | p6)) & (Equivalent(p8, p4 | p7)) & (Equivalent(p9, p3 | p8))

In [32]:
to_cnf(tseitin1((x0 & x1) | (x2 & x3) | (x4 & x5) | (x6 & x7)| (x8 & x9)| (x10 & x11)))

p11 & (p10 | ~p2) & (p10 | ~p9) & (p11 | ~p1) & (p11 | ~p10) & (p7 | ~p5) & (p7 | ~p6) & (p8 | ~p4) & (p8 | ~p7) & (p9 | ~p3) & (p9 | ~p8) & (x0 | ~p1) & (x1 | ~p1) & (x10 | ~p2) & (x11 | ~p2) & (x2 | ~p3) & (x3 | ~p3) & (x4 | ~p4) & (x5 | ~p4) & (x6 | ~p5) & (x7 | ~p5) & (x8 | ~p6) & (x9 | ~p6) & (p1 | p10 | ~p11) & (p2 | p9 | ~p10) & (p3 | p8 | ~p9) & (p4 | p7 | ~p8) & (p5 | p6 | ~p7) & (p1 | ~x0 | ~x1) & (p2 | ~x10 | ~x11) & (p3 | ~x2 | ~x3) & (p4 | ~x4 | ~x5) & (p5 | ~x6 | ~x7) & (p6 | ~x8 | ~x9)